# Finding and Ordering Imagery for the Largest Active Fire on Earth

In this notebook, we'll combine NASA's FIRMS (Fire Information for Resource Management System) active fire data with Google Earth Engine and Planet imagery to:

1. **Load today's active fires** from the MODIS or VIIRS instruments via FIRMS
2. **Identify the largest fire** by grouping nearby fire pixels into connected clusters
3. **Polygonize the fire boundary** to create an area of interest (AOI)
4. **Search Planet's catalog** for recent PlanetScope imagery over that AOI
5. **Order and visualize** the imagery directly in this notebook using geemap

This workflow demonstrates real-world remote sensing analysis: detecting an event from coarse hotspot data, defining a precise AOI, and acquiring high-resolution commercial imagery for detailed assessment.

## APIs and Tools

- **[Google Earth Engine Python API](https://developers.google.com/earth-engine/guides/python_install)** – for geospatial analysis and data processing
- **[geemap](https://geemap.org/)** – interactive mapping library for Earth Engine in Python
- **[Planet SDK for Python](https://planet-sdk-for-python-v2.readthedocs.io/)** – for programmatic access to Planet's data catalog
- **[Planet Orders API](https://developers.planet.com/docs/apis/orders/)** – for ordering and delivering Planet imagery

## Datasets

- **[FIRMS Active Fire Data](https://firms.modaps.eosdis.nasa.gov/)** – near real-time fire detections from MODIS and VIIRS instruments
- **[PlanetScope Imagery](https://developers.planet.com/docs/data/planetscope/)** – high-resolution (3–5 m) daily imagery from Planet's satellite constellation

## Prerequisites

- Authenticated access to Google Earth Engine (`ee.Authenticate()` and `ee.Initialize()`)
- A Planet API key (sign up at [planet.com](https://www.planet.com/) for a free trial or educational account)
- Python packages: `earthengine-api`, `geemap`, `planet`, `requests`, `geopandas`, `shapely`

Let's get started!

## Installing Required Libraries

Before we begin, we need to install the necessary Python packages for this workflow:

- **earthengine-api** – Google Earth Engine Python API for geospatial analysis
- **geemap** – interactive mapping library that simplifies Earth Engine visualization
- **planet** – Planet SDK for searching and ordering commercial satellite imagery
- **requests** – HTTP library for API calls (e.g., downloading FIRMS data)
- **geopandas** – geospatial extension of pandas for working with vector data
- **shapely** – geometric operations library for creating and manipulating spatial objects

Run the cell below to install or upgrade these packages. The `-q` flag suppresses verbose output, and `--upgrade` ensures you have the latest versions.

**Note:** Some operations in this workflow (particularly Planet API interactions) may benefit from asynchronous execution to handle multiple requests efficiently.

In [1]:
# Install all required Python packages for this notebook
# The -q flag keeps the output quiet (less verbose)
# The --upgrade flag ensures we get the latest versions of each package

!pip install -q --upgrade earthengine-api leafmap geemap planet requests geopandas shapely

## Imports

In [2]:
# Core Earth Engine API for accessing and processing geospatial datasets
import ee

# geemap wraps Earth Engine objects for interactive mapping in notebooks
import geemap as geemap

# Planet SDK pieces:
# Auth: used to manage API key sessions
# DataClient: search/catalog queries (e.g., filtering for recent PlanetScope imagery)
# OrdersClient: place orders (e.g., bundle, clip, deliver assets)
from planet import Auth, DataClient, OrdersClient

# requests for simple HTTP downloads (e.g., FIRMS CSV if needed outside EE)
import requests

# geopandas for handling vector data (GeoDataFrame) locally once exported or downloaded
import geopandas as gpd

# shapely provides geometric operations (buffer, area, bounds, etc.)
from shapely.geometry import Polygon, shape, mapping

# Optional: set up Planet authentication (expects your API key in the PL_API_KEY env var)
# auth = Auth.from_env()
# pl_session = auth.create_client_session()
# data_client = DataClient(pl_session)
# orders_client = OrdersClient(pl_session)

# Initialize Earth Engine (authentication handled in a later cell)
# ee.Initialize()


## 1. Authentication with Google Earth Engine

Before you can use Google Earth Engine (GEE), you need to authenticate your account and initialize the API. There are several ways to do this:

*   **Browser Authentication (Recommended for Beginners):** This is the easiest method. GEE will open a browser window to allow you to log in with your Google account.
*   **Service Account:** More suitable for automated scripts and server-side applications. Requires creating a service account in the Google Cloud Console and downloading credentials.

### 1.1 Browser Authentication

The following code snippet initializes the Earth Engine API and authenticates your account using browser authentication. Run this cell first to authenticate.

**What happens when you run this cell:**

1. The code tries to initialize Earth Engine with `ee.Initialize()`
2. If you haven't authenticated yet, it will fail and catch the error
3. It then runs `ee.Authenticate()` which opens your web browser
4. You'll log in with your Google account and grant permissions
5. After successful authentication, it runs `ee.Initialize()` again to connect to Earth Engine
6. Finally, it prints a confirmation message

**Note:** You only need to authenticate once per computer. After that, you can just run `ee.Initialize()` in future sessions.

In [3]:
    ee.Authenticate()
    ee.Initialize(project='greenai-hackathon-maples-two')

In [4]:
# Load the FIRMS image collection from Earth Engine's public data catalog
dataset = ee.ImageCollection("FIRMS")

# Create a rectangle covering the entire world (longitude: -180 to 180, latitude: -90 to 90)
geometry = ee.Geometry.Rectangle([-180, -90, 180, 90])

# Sort the collection by date (newest first) and get the most recent image
# This gives us today's fire detections
lastimg = dataset.sort('system:time_start', False).first()

# Select the T21 band (brightness temperature) and keep only pixels hotter than 100°C
# gt(100) means "greater than 100" – this creates a binary image where fires = 1, non-fires = 0
fires = lastimg.select('T21').gt(100)

# Define visualization colors for fire temperature
# Cooler fires appear yellow, hotter fires appear red
firesVis = {
    "min": 100.0,      # Minimum temperature to display (in °C)
    "max": 500.0,      # Maximum temperature to display (in °C)
    "palette": ["yellow", "orange", "red"]  # Color ramp from cool to hot
}

# Create an interactive map centered on the world
m = geemap.Map(center=[0, 0], zoom=2)

# Add the fire temperature layer to the map
m.addLayer(lastimg.select('T21'), firesVis, 'FIRMS T21')

# Display the map (this must be the last line to show the map in the notebook)
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…

In [5]:
# Step 1: Count how many fire pixels are connected to each other
# connectedPixelCount looks at all neighboring pixels (up/down/left/right/diagonals)
# and counts how many fire pixels form a single cluster
# The number 1000 is the maximum cluster size we'll track
# True means we use 8-directional connectivity (includes diagonals)
connectedCount = fires.connectedPixelCount(1000, True)

# Step 2: Define how to color the clusters on the map
# Small clusters will appear purple, large clusters will appear yellow/gold
conn_vis = {
    'min': 0,          # Smallest cluster size (0 pixels)
    'max': 500,        # Largest cluster size we expect (500 pixels)
    'palette': ['#4B0082', '#6A5ACD', '#8A2BE2', '#DA70D6', '#FFD700']  # Purple to yellow
}

# Step 3: Calculate the minimum and maximum cluster sizes across the entire world
# This tells us the range of fire cluster sizes detected today
minMax = connectedCount.reduceRegion(
    reducer=ee.Reducer.minMax(),  # Find both min and max values
    geometry=geometry,             # Search the entire world (defined earlier)
    scale=1000,                    # Use 1 km resolution for the calculation
    maxPixels=1e9                  # Allow up to 1 billion pixels to be processed
)

# Step 4: Print the results so we can see the smallest and largest cluster sizes
print("Min and Max values:", minMax.getInfo())

# Step 5: Add the cluster layer to the map
# Each fire pixel is now colored by its cluster size
m.addLayer(connectedCount, conn_vis, 'Connected pixels (count) — purple→yellow')

# Step 6: Display the map
# This shows the fire temperature AND the cluster sizes together
m

Min and Max values: {'T21_max': 238, 'T21_min': 1}


Map(bottom=19699.0, center=[-31.419288124288357, 21.978149414062504], controls=(WidgetControl(options=['positi…

In [7]:
# Create a new map centered on the world
# We'll add layers showing the connected fire clusters
m = geemap.Map(center=[0, 0], zoom=2)

# Add the original fire temperature data (T21 band from FIRMS)
# Yellow = cooler fires, red = hotter fires
m.addLayer(
    lastimg.select('T21'), 
    firesVis, 
    'FIRMS T21'
)

# Add the connected cluster visualization
# Purple = small clusters (few connected pixels)
# Yellow = large clusters (many connected pixels)
m.addLayer(
    connectedCount, 
    conn_vis, 
    'Connected pixels (count) — purple→yellow'
)

# Display the map
# This should be the last line so the map appears in the notebook output
m


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…

In [8]:
# Extract the maximum cluster size from the minMax dictionary.
# This tells us how many connected pixels are in the largest fire cluster.
T21_max = ee.Number(minMax.get('T21_max'))

# Create a binary image where pixels equal to T21_max are set to 1, all others to 0.
# This isolates only the largest fire cluster from all the fire pixels.
largest_cluster = connectedCount.eq(T21_max)

# Apply a mask to show only the largest cluster (hide pixels with value 0).
# updateMask() makes non-matching pixels transparent in visualizations.
largest_mask = largest_cluster.updateMask(largest_cluster)

# Add the largest cluster to the map as a red layer for visual confirmation.
# The palette maps the value 1 (our cluster) to red.
m.addLayer(largest_mask, {'min': 0, 'max': 1, 'palette': ['red']}, 'Largest cluster (count == T21_max)')

# Print the maximum cluster size to the console (fetch from server to client).
# This shows us exactly how many connected fire pixels are in the largest cluster.
print('T21_max:', T21_max.getInfo())

# Print the full minMax dictionary for reference (shows both min and max values).
print("Min and Max values:", minMax.getInfo())

T21_max: 238
Min and Max values: {'T21_max': 238, 'T21_min': 1}


In [9]:

# Ensure a geemap Map object exists (create one if needed)
try:
    m  # noqa: F821
except NameError:
    # Use the existing import from earlier cells; do not re-import
    m = geemap.Map(center=[0, 0], zoom=2)

# Polygonize the largest masked cluster and zoom to it
vectors = largest_mask.reduceToVectors(
    reducer=ee.Reducer.countEvery(),
    geometry=geometry,
    scale=1000,
    maxPixels=1e9,
    eightConnected=True,
    geometryType='bb'
)

# Compute area for each polygon and select the largest.
# Specify a non-zero maxError to avoid Geometry.area errors in server-side ops.
def set_area(feat):
    geom = feat.geometry()
    # use a small non-zero maxError (in meters)
    area_m = geom.area(1)
    return feat.set('area', area_m)

vectors = vectors.map(set_area)
largest_feature = ee.Feature(vectors.sort('area', False).first())

# Compute and print the area of the largest_feature
area_m = largest_feature.geometry().area(1)  # area in square meters (server-side)
area_m_val = area_m.getInfo()  # fetch to client
print('Largest feature area (m²):', area_m_val)
print('Largest feature area (km²):', area_m_val / 1e6)

# Add layers and center the map on the largest polygon
m.addLayer(vectors, {'color': 'orange'}, 'Largest cluster polygons')
m.addLayer(ee.FeatureCollection(largest_feature), {'color': 'red'}, 'Largest polygon')
m.centerObject(largest_feature, 10)
m

Largest feature area (m²): 1043455464.6984392
Largest feature area (km²): 1043.4554646984393


Map(bottom=812.0, center=[-21.650714577843978, 128.1507440474627], controls=(WidgetControl(options=['position'…

In [9]:
import json  # Standard module for working with JSON data

# Wrap the Earth Engine Feature in a FeatureCollection so it matches GeoJSON expectations.
feature_collection_geojson = {
    "type": "FeatureCollection",
    "features": [largest_feature.getInfo()]  # Bring the server-side feature to the client as a dictionary.
}

# Pretty-print the GeoJSON so new programmers can inspect its structure.
print(json.dumps(feature_collection_geojson, indent=2))

{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "geometry": {
        "geodesic": false,
        "type": "Polygon",
        "coordinates": [
          [
            [
              19.440129745332264,
              -34.69081973540713
            ],
            [
              19.839461375655798,
              -34.69081973540713
            ],
            [
              19.839461375655798,
              -34.57386883739649
            ],
            [
              19.440129745332264,
              -34.57386883739649
            ],
            [
              19.440129745332264,
              -34.69081973540713
            ]
          ]
        ]
      },
      "id": "+21827+13864",
      "properties": {
        "area": 475128062.5442215,
        "count": 231,
        "label": 1
      }
    }
  ]
}


## Export to geojson

In [10]:
from pathlib import Path  # Standard library helper for filesystem paths

# Ensure the output directory exists (creates ./output/ if needed).
output_dir = Path("output")
output_dir.mkdir(parents=True, exist_ok=True)

# Specify the GeoJSON file path inside the output directory.
geojson_path = output_dir / "largest_fire_cluster_1-9-2026.geojson"

# Serialize the FeatureCollection dictionary to GeoJSON with nice formatting.
with geojson_path.open("w", encoding="utf-8") as fp:
    json.dump(feature_collection_geojson, fp, ensure_ascii=False, indent=2)

# Provide a quick confirmation of where the file was written.
print(f"GeoJSON exported to: {geojson_path.resolve()}")

GeoJSON exported to: /Users/maples/GitHub/google_earth_engine_python_101/notebooks/output/largest_fire_cluster_1-9-2026.geojson
